In [23]:
import pandas as pd
#Python library for frequent pattern mining and association rule learning.
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

C:\Users\Jagadish\AppData\Local\Temp\Temp1_Course.zip\Course Files[DS]\Associate Rules Script.ipynb

In [24]:
# reads an Excel file named 'online_retail_II.xlsx' into a pandas DataFrame called 'df'. 
df = pd.read_excel('B:downlods/online_retail_II.xlsx')
df.head()#first five values

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [25]:
df['Description'] = df['Description'].str.strip()# removes white space from the 'Description' column
print(df['Description'])
df.dropna(axis=0, subset=['Invoice'], inplace=True)#drops any rows from the 'Invoice' column contains missing values (NaNs).
df['Invoice'] = df['Invoice'].astype('str')#converts the 'Invoice' column in the DataFrame 'df' to a string data type
df = df[~df['Invoice'].str.contains('C')]# selects rows that do not contain the letter 'C'.'C' represents cancelled orders
print(df)

0         15CM CHRISTMAS GLASS BALL 20 LIGHTS
1                          PINK CHERRY LIGHTS
2                         WHITE CHERRY LIGHTS
3                 RECORD FRAME 7" SINGLE SIZE
4              STRAWBERRY CERAMIC TRINKET BOX
                         ...                 
525456                   FELTCRAFT DOLL ROSIE
525457           FELTCRAFT PRINCESS LOLA DOLL
525458         FELTCRAFT PRINCESS OLIVIA DOLL
525459     PINK FLORAL FELTCRAFT SHOULDER BAG
525460                 JUMBO STORAGE BAG SUKI
Name: Description, Length: 525461, dtype: object
       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041          RECORD FRAME 7" SINGLE SIZE        48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        2

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515255 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      515255 non-null  object        
 1   StockCode    515255 non-null  object        
 2   Description  512324 non-null  object        
 3   Quantity     515255 non-null  int64         
 4   InvoiceDate  515255 non-null  datetime64[ns]
 5   Price        515255 non-null  float64       
 6   Customer ID  407695 non-null  float64       
 7   Country      515255 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.4+ MB


In [27]:
# DataFrame 'basket' should have one row for each unique invoice,
# and one column for each unique product purchased by customers in France. 
basket = (df[df['Country'] =="France"]# selects only the rows where the 'Country' column equals 'France'.
          .groupby(['Invoice', 'Description'])['Quantity']# groups the selected DataFrame by 'Invoice' and 'Description' columns and sums up the 'Quantity'
          .sum().unstack().reset_index().fillna(0)#each unique product becomes a separate column and 'Invoice' becomes a regular column.
          .set_index('Invoice'))
print(basket)

Description  10 COLOUR SPACEBOY PEN  12 IVORY ROSE PEG PLACE SETTINGS  \
Invoice                                                                 
489439                          0.0                               0.0   
489557                          0.0                               0.0   
489883                          0.0                               0.0   
490139                          0.0                               0.0   
490152                          0.0                               0.0   
...                             ...                               ...   
537693                          0.0                               0.0   
537897                          0.0                               0.0   
537967                          0.0                               0.0   
538008                          0.0                               0.0   
538093                          0.0                               0.0   

Description  12 MESSAGE CARDS WITH ENVELOPES  12 P

In [28]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)#remove the 'POSTAGE' column as it represents a shipping cost
print(basket_sets)

Description  10 COLOUR SPACEBOY PEN  12 IVORY ROSE PEG PLACE SETTINGS  \
Invoice                                                                 
489439                            0                                 0   
489557                            0                                 0   
489883                            0                                 0   
490139                            0                                 0   
490152                            0                                 0   
...                             ...                               ...   
537693                            0                                 0   
537897                            0                                 0   
537967                            0                                 0   
538008                            0                                 0   
538093                            0                                 0   

Description  12 MESSAGE CARDS WITH ENVELOPES  12 P

In [29]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
print(frequent_itemsets)

     support                                           itemsets
0   0.116183                       (ASSORTED COLOUR MINI CASES)
1   0.091286                     (BAKING SET 9 PIECE RETROSPOT)
2   0.091286               (CHILDS BREAKFAST SET CIRCUS PARADE)
3   0.116183                          (CIRCUS PARADE LUNCH BOX)
4   0.087137                             (DOLLY GIRL LUNCH BOX)
..       ...                                                ...
57  0.070539  (WOODLAND CHARLOTTE BAG, RED SPOTTY CHARLOTTE ...
58  0.078838  (RED/WHITE DOT MINI CASES, RED TOADSTOOL LED N...
59  0.087137  (ROUND SNACK BOXES SET OF4 WOODLAND, RED TOADS...
60  0.095436  (ROUND SNACK BOXES SET OF4 WOODLAND, ROUND SNA...
61  0.099585  (SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...

[62 rows x 2 columns]


c:\Users\Jagadish\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [30]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CIRCUS PARADE LUNCH BOX),(PLASTERS IN TIN CIRCUS PARADE),0.116183,0.161826,0.070539,0.607143,3.751832,0.051738,2.133535
1,(PLASTERS IN TIN CIRCUS PARADE),(CIRCUS PARADE LUNCH BOX),0.161826,0.116183,0.070539,0.435897,3.751832,0.051738,1.566767
2,(SPACEBOY LUNCH BOX),(CIRCUS PARADE LUNCH BOX),0.132780,0.116183,0.087137,0.656250,5.648438,0.071710,2.571105
3,(CIRCUS PARADE LUNCH BOX),(SPACEBOY LUNCH BOX),0.116183,0.132780,0.087137,0.750000,5.648437,0.071710,3.468880
4,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.132780,0.087137,0.070539,0.531250,6.096726,0.058969,1.947441


In [31]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.087137,0.13278,0.070539,0.809524,6.096726,0.058969,4.552905
34,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.120332,0.13278,0.099585,0.827586,6.232759,0.083607,5.029876


 invoice is a document that lists the goods or services that a customer has purchased from a supplier, along with the prices and any applicable taxes or fees. An invoice typically includes a unique invoice number, the date of purchase, and information about the customer and supplier.